In [46]:
import pandas as pd
import glob
import numpy as np

In [47]:
df = pd.read_csv("./new_book_1.csv")
df = df.drop(['Unnamed: 0'], axis=1)
df = df.dropna()
df = df.reset_index(drop=True)
df = df.drop_duplicates(subset="title")
df = df.reset_index(drop=True)

In [48]:
df

,title,desc,img
0,"HTML, CSS, SCRATCH, PYTHON. Моя первая книга п...",Описание\nХотите научиться создавать программы...,https://cdn1.ozone.ru/s3/multimedia-q/wc200/60...
1,"Учим Python, делая крутые игры | Свейгарт Эл",Описание\nНе играй в игры - создавай их! Эта к...,https://cdn1.ozone.ru/s3/multimedia-j/wc1200/6...
2,Изучаем C++ через программирование игр Доусон ...,Описание\nЕсли вы хотите научиться программиро...,https://cdn1.ozone.ru/s3/multimedia-6/wc1200/6...
3,"Язык программирования C | Ритчи Деннис М., Кер...","Описание\nКлассическая книга по языку C, напис...",https://cdn1.ozone.ru/multimedia/wc1200/102799...
4,Основы Python. Научитесь думать как программис...,"Описание\n""Практическое руководство по основам...",https://cdn1.ozone.ru/s3/multimedia-0/wc1200/6...
...,...,...,...
150,C/C++. Структурное программирование. Практикум,Описание\nВ практикуме на примерах рассматрива...,https://cdn1.ozone.ru/s3/multimedia-s/wc1200/6...
151,Разработка приложений под iPhone. Полное руков...,Описание\nНаучиться писать приложения для iPho...,https://cdn1.ozone.ru/multimedia/wc1200/100214...
152,С/С++. Процедурное программирование,Описание\nПодробно рассмотрены процедурные воз...,https://cdn1.ozone.ru/multimedia/wc1200/100766...
153,Профессиональная разработка на Python,Описание\nВ книге описаны современные передовы...,https://cdn1.ozone.ru/s3/multimedia-y/wc1200/6...


In [49]:
from tqdm.contrib.concurrent import process_map  # or thread_map

In [50]:
from nltk.corpus import stopwords
import re
def clean_dataset(senteces):
    text = senteces.lower()
    text = re.findall(r'\w+', text)
    tokenize = [i for i in text if not i.isdigit()]
    tokenize = [word for word in tokenize if not word in stopwords.words()]
    text = ' '.join(tokenize)
    return text

In [51]:
desc_book = process_map(clean_dataset, list(df['desc']), max_workers=6)

  0%|          | 0/155 [00:00<?, ?it/s]

In [52]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('quora-distilbert-multilingual')

In [53]:
embeddings_course = model.encode(desc_book)

In [54]:
df['embd'] = list(embeddings_course)

In [55]:
df = df.drop(['desc'], axis=1)

In [56]:
dat = df.to_dict()

In [57]:
import pickle

In [58]:
with open('./dat_book.pickle', 'wb') as f:
    pickle.dump(dat, f)

In [59]:
data_hh_list = glob.glob("./hh_data/*.csv")
df_bd = pd.DataFrame(columns=['Ids', 'Employer', 'Name', 'Salary', 'From', 'To', 'Experience',
       'Schedule', 'Keys', 'Description'])
for csv in data_hh_list:
    df_bd = df_bd.append(pd.read_csv(csv))
df_bd = df_bd.reset_index(drop=True)

front = df_bd.loc[df_bd['Name'].str.contains("ios", case=False)]

front = front.reset_index(drop=True)
front

,Ids,Employer,Name,Salary,From,To,Experience,Schedule,Keys,Description
0,44691569,FaceOut,IOS разработчик в стартап,False,NaN,NaN,От 3 до 6 лет,Удаленная работа,"['Разработка ПО', 'Scrum', 'Start-up project',...",Мы – опытная команда фаундеров из Кремниевой Д...
1,44725803,FaceOut,Мобильный разработчик iOS & Android,False,NaN,NaN,От 3 до 6 лет,Удаленная работа,"['Android', 'Разработка ПО', 'Android SDK', 'S...",Мы – опытная команда фаундеров из Кремниевой Д...
2,44965540,Mail.ru Group,IOS-разработчик (Почта),False,NaN,NaN,От 3 до 6 лет,Полный день,['iOS'],Мобильная Почта стартовала в 2012 году и включ...
3,45193826,Wheely,"Senior iOS Developer, Chauffeur Experience",False,NaN,NaN,От 3 до 6 лет,Полный день,"['Swift', 'Reactive Swift', 'Cocoapods']",Wheely is a ride-hailing app recognized for it...
4,44128979,Авито,IOS разработчик (Юнит Speed),False,NaN,NaN,От 3 до 6 лет,Полный день,"['Mac Os', 'Swift', 'CI/CD', 'CI', 'UI']","Мы бережём здоровье сотрудников, поэтому на вр..."
...,...,...,...,...,...,...,...,...,...,...
606,45381832,Туту.ру,IOS-разработчик в продукт Отели,False,NaN,NaN,От 1 года до 3 лет,Полный день,"['Git', 'Управление проектами', 'Atlassian Jir...","Tutu.ru — онлайн-сервис для путешествий, котор..."
607,45380613,Туту.ру,IOS-разработчик в команду выдачи,False,NaN,NaN,От 1 года до 3 лет,Полный день,"['Git', 'Rx', 'Atlassian Jira', 'Swift', 'Coco...","Транспортная выдача Туту.ру — это весь слой, с..."
608,45312779,iHerb,Senior iOS Developer,False,NaN,NaN,От 3 до 6 лет,Полный день,"['Objective-C', 'Swift', 'iOS', 'JSON API', 'R...",Job Summary: The Sr. iOS Developer will work d...
609,45297811,Luxoft,IOS разработчик,False,NaN,NaN,От 1 года до 3 лет,Удаленная работа,"['iOS', 'Objective-C', 'Swift']","Второй по величине банк в стране, приложение..."


In [60]:
desc = list(front['Description'])

In [61]:
r = process_map(clean_dataset, desc, max_workers=6)

  0%|          | 0/611 [00:00<?, ?it/s]

In [62]:
embeddings_vac = model.encode(r)

In [63]:
front_embeding = embeddings_vac[608]

In [64]:
from scipy.spatial.distance import cosine, correlation, dice, minkowski, hamming

In [65]:
rec_book = []
for bk_index in range(len(embeddings_course)):
    rec_book.append(cosine(front_embeding, embeddings_course[bk_index]))

In [66]:
for rec in sorted(enumerate(rec_book), key=lambda x: x[1], reverse=True)[:10]:
    print(dat['title'][rec[0]])

Искусство программирования, том 4А. Комбинаторные алгоритмы. Часть 1 | Кнут Дональд Эрвин
Эффективное использование С++. 55 верных способов улучшить структуру и код ваших программ | Мейерс Скотт
Как делать деньги, не выходя из дома. С пошаговым планом заработка 1000 в месяц
Шаблоны C++. Справочник разработчика | Джосаттис Николаи М., Грегор Дуглас
Служащие. Из жизни современной Германии | Кракауэр Зигфрид, Мичковский Олег И.
Modern X86 Assembly Language Programming. Covers x86 64-bit, AVX, AVX2, and AVX-512
Прогулки по Армении | Нет автора
Practical C++ Design. From Programming to Architecture
Квантовые вычисления. Прикладной подход
Язык программирования C# 7 и платформы. NET и NET Core. 8-е изд
